In [155]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast
import math
import random

In [245]:
preferences = pd.read_csv('Fall_clinical_placements.csv')
preferences = preferences[preferences.columns[:8]].copy()

display(preferences)
def string_to_num(choice):
    if choice == 'First Choice':
        return 1
    elif choice == 'Second Choice':
        return 2
    else:
        return 3
    
preferences.mask(preferences == 'First Choice', 1 , inplace=True )
preferences.mask(preferences == 'Second Choice', 2 , inplace=True )
preferences.mask(preferences == 'Third Chice', 3 , inplace=True )
preferences.mask(preferences == 'Third Choice', 3 , inplace=True )
preferences.mask(preferences == 'third Chice', 3 , inplace=True )
preferences.mask(preferences == 'Med Surg', '[1,2]', inplace=True)
preferences.mask(preferences == 'Psych', '[2,1]', inplace=True)

print(preferences.columns)
preferences.columns = ['Name', 'r', '[0,0]', '[0,1]', '[0,2]', '[1,0]', '[1,1]', '[1,2]']


,Your Name,First Preference - Clinical Specialty,Clinical Site Med Surg - Mondays Cedar Sinai Medical Center,Clinical Site Med Surg - Fridays Valley Presbyterian Medical Center,Clinical Site Med Surg - Saturdays PIH Whittier Medical Center,Clinical Site Psych - Mondays LADMC Ingleside,Clinical Site Psych - Thursdays Gateways Hospital and Mental Health Center,Clinical Site Psych - Saturdays Gateways Hospital and Mental Health Center
0,Alan Felix,Med Surg,First Choice,Third Chice,Second Choice,Second Choice,Second Choice,Third Choice
1,Amanda Yannuzzi,Med Surg,First Choice,Third Chice,Second Choice,Second Choice,First Choice,Third Choice
2,Anita Li,Med Surg,Second Choice,First Choice,Third Chice,Third Choice,First Choice,Second Choice
3,Anne Crawford,Med Surg,First Choice,Second Choice,Third Chice,First Choice,Second Choice,Third Choice
4,Anthony Piscitella,Med Surg,Second Choice,First Choice,Third Chice,First Choice,Second Choice,Third Choice
5,Avi Spiegel,Med Surg,First Choice,Second Choice,Third Chice,Third Choice,First Choice,Second Choice
6,Derek Iwata,Med Surg,Second Choice,First Choice,Third Chice,First Choice,Second Choice,Third Choice
7,Elise,Psych,Third Chice,First Choice,Second Choice,First Choice,Second Choice,Third Choice
8,Elissa Braitman,Med Surg,First Choice,Third Chice,Second Choice,Third Choice,First Choice,Second Choice
9,Elizabeth Lim,Med Surg,First Choice,Second Choice,Third Chice,Second Choice,First Choice,Third Choice


Index(['Your Name', 'First Preference - Clinical Specialty',
       'Clinical Site Med Surg - Mondays Cedar Sinai Medical Center',
       'Clinical Site Med Surg - Fridays Valley Presbyterian Medical Center',
       'Clinical Site Med Surg - Saturdays PIH Whittier Medical Center',
       'Clinical Site Psych - Mondays LADMC Ingleside',
       'Clinical Site Psych - Thursdays Gateways Hospital and Mental Health Center',
       'Clinical Site Psych - Saturdays Gateways Hospital and Mental Health Center'],
      dtype='object')


In [281]:
preferences.sort_values('Name',inplace=True)

In [248]:
# Preparing data

N = 2 # num rotations
M = 3 # num placements
num_students = preferences.shape[0]
S = np.zeros((num_students,N,M))
for col in preferences.columns[2:]: 
    # Converting string to list
    rotation_idx = ast.literal_eval(col)
    S[:,rotation_idx[0],rotation_idx[1]] = preferences[col]

r = np.zeros((num_students, 2))
for i in range(num_students):
    r[i,] = ast.literal_eval(preferences['r'][i])
    
pp = np.zeros((num_students, 1))
pp[[9,11,13,18,26]] = 1


print(f'S: {S.shape}')
print(f'r: {r.shape}')
print(f'pp: {pp.shape}')

S: (29, 2, 3)
r: (29, 2)
pp: (29, 1)


In [250]:
restrictions = np.asarray([
    [[1, 0], # Monday Med Surg (0,0)
     [], # Friday Med Surg (0,1)
     [1,2]], # Saturday Med Surg (0,2)
    [[(0,0)], # Monday Psych (1,0)
     [], # Thursday Psych (1,1)
     [0,2]] # Saturday Psych (1,2)
], dtype=object)

restrictions


array([[list([1, 0]), list([]), list([1, 2])],
       [list([(0, 0)]), list([]), list([0, 2])]], dtype=object)

In [251]:
preference_list=np.zeros((num_students * N * M, ), dtype=int)
preference_list_meta_data = np.zeros((num_students*N*M, 3),dtype=int)
i = 0
for k in range(num_students):
    S_prime_k = S_prime[k]
    for row in range(N):
        for col in range(M):
            preference_list_meta_data[i] = [k, row, col] 
            preference_list[i] = S_prime_k[row,col]
            i += 1              

In [252]:
(preference_list_meta_data[:,0] == 1) & (preference_list_meta_data[:,1] == 1)

array([False, False, False, False, False, False, False, False, False,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [253]:
np.argsort(preference_list)

array([161, 134,  95,  29,  59,  33,  87,  81,  23, 140,  42,  77, 155,
        69,  51,  65, 122,  41,  99, 149, 116, 165, 111, 105,   5,  15,
        11, 129, 170,  75,  74, 143, 100, 126,  68, 146, 147, 119,  63,
        62, 120,  71, 128, 113, 114, 131, 133, 104,  96,  93,  92, 141,
       107, 137,  89, 138,  84,  82,  80, 109, 173, 125,  57, 159,  35,
        32,  26, 164,  22,  20, 158,  17, 167,   9, 169,   7,   4,   3,
         1,  14,  38,  28,  40,  56,  44, 152,  47,  53,  49, 153,  16,
       106,  70, 130, 112,  10,  52, 115,  64, 121, 168, 166, 132, 101,
        83,  43, 139,  34,  88,  76,  39, 160,  27,  94, 154,  21,  58,
       148, 135, 157, 163, 151, 171, 145,  86, 123,  55,  61,  67,  36,
        31,  73,  79,  46,  24,  19,  50,  91,   2, 117,   8, 103, 110,
        12,  98, 162,  18,  37,   6,  45, 156,  13,  30,  25,  60,  54,
       118, 127, 108, 102,  97, 136,  90,  48, 172,  78, 142, 144,  72,
        66, 124, 150,  85,   0])

In [309]:
def place(preferences, preferences_meta):
    placement = np.zeros((num_students,N))
    random.shuffle(preferences)
    sort_idxs = np.argsort(preferences)
    sorted_preferences = preferences[sort_idxs]
    
    sorted_meta = preferences_meta[sort_idxs]
    sorted_preferences = sorted_preferences[::-1]
    sorted_meta = sorted_meta[::-1]
    print(sorted_preferences)
    placement_count = np.zeros((N,M)) + 10
    placements = {}
    score = 0
    while sorted_preferences.shape[0] > 0:
        current_student, current_rotation, current_placement = sorted_meta[0]
        print(current_rotation)
        print(current_placement)
        current_conflicts = restrictions[current_rotation, current_placement]
        
        # Check if current placement is available
        if placement_count[current_rotation, current_placement] > 0:
        
            # Add score
            score += sorted_preferences[0]
            
            # Place student in preferred location
            if f'[{current_rotation},{current_placement}]' in placements:
                placements[f'[{current_rotation},{current_placement}]'].append(current_student)
            else:
                placements[f'[{current_rotation},{current_placement}]'] = [current_student]
                
            # Update number of placements available
            placement_count[current_rotation, current_placement] += (-1)
            
            # Then remove current student from the list (for current rotation)
            print(f'preferences before: {sorted_preferences.shape}')
            print(f'meta before: {sorted_meta.shape}')
            removal_mask = np.invert(((sorted_meta[:,0] == current_student) & (sorted_meta[:,1] == current_rotation)))
            print(removal_mask)
            sorted_preferences = sorted_preferences[removal_mask]
            print(f'preferences after: {sorted_preferences.shape}')
            sorted_meta = sorted_meta[removal_mask]
            print(f'meta after: {sorted_meta.shape}')
            
        else:
            # Remove current entry and keep going
            sorted_preferences = sorted_preferences[1:]
            sorted_meta = sorted_meta[1:]

    return placements, score
        
        
        
    

In [310]:
placements, score = place(preference_list, preference_list_meta_data)

[ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3
 -3 -3 -3 -3 -3 -3]
1
2
preferences before: (174,)
meta before: (174, 3)
[False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [311]:
placements

{'[1,2]': [28, 19, 1, 10, 12, 9, 20, 26, 13],
 '[1,1]': [25, 5, 3, 23, 16, 8, 21, 15, 14, 0],
 '[0,0]': [25, 19, 20, 5, 28, 26, 24, 11, 18],
 '[0,1]': [7, 23, 17, 8, 14, 3, 15, 1, 12, 21],
 '[1,0]': [24, 18, 6, 4, 7, 27, 22, 17, 11, 2],
 '[0,2]': [22, 2, 16, 13, 9, 6, 4, 0, 27, 10]}

In [312]:
score

54

In [300]:
actual_placements = pd.read_csv('actual_fall_clinical_placements.csv')
actual_placements = actual_placements[actual_placements.columns[:-1]]
actual_placements.fillna(0, inplace=True)

actual_placements.mask(actual_placements == 'PIH', 3 , inplace=True )
actual_placements.mask(actual_placements == 'VPH', 2 , inplace=True )
actual_placements.mask(actual_placements == 'Cedars', 1 , inplace=True )
actual_placements.mask(actual_placements == 'Mon LADMC', 1 , inplace=True )
actual_placements.mask(actual_placements == 'MON LADMC', 1 , inplace=True )
actual_placements.mask(actual_placements == 'Thurs Gateways', 2 , inplace=True )
actual_placements.mask(actual_placements == 'SAT GATEWAYS', 3 , inplace=True )


actual_placements = actual_placements.sort_values('Summer preference points').reset_index(drop=True).reset_index()
display(actual_placements.head())
student_list = np.asarray(actual_placements['index'])
med_surg_placement = np.asarray(actual_placements['Med/Surg Choice 1'])
psych_placement = np.asarray(actual_placements['Psych Choice 1'])



,index,Name,Summer preference points,Med/Surg Choice 1,Psych Choice 1
0,0,Felix,Alan,3,1
1,1,Yannuzzi,Amanda,1,3
2,2,Li,Anita,2,1
3,3,Crawford,Anne,1,3
4,4,Piscitella,Anthony,2,3


In [301]:
student_list

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

In [303]:
med_surg_placement

array([3, 1, 2, 1, 2, 1, 3, 2, 1, 1, 2, 1, 2, 1, 2, 2, 3, 3, 1, 3, 2, 2,
       3, 2, 1, 3, 1, 3, 3], dtype=object)

In [304]:
psych_placement

array([1, 3, 1, 3, 3, 2, 1, 1, 3, 3, 1, 2, 1, 3, 3, 3, 1, 1, 3, 2, 2, 3,
       2, 2, 2, 2, 2, 1, 2], dtype=object)

In [274]:
actual_placements

,Name,Summer preference points,Med/Surg Choice 1,Psych Choice 1,Spring Preference Points
0,Lopez,Leslie,PIH,Mon LADMC,3.0
1,Harmon,Taylor,PIH,Mon LADMC,3.0
2,Dao,Jocelyn,PIH,Mon LADMC,2.5
3,Iwata,Derek,PIH,Mon LADMC,2.0
4,Manzo,Stephanie,PIH,Thurs Gateways,2.0
5,Felix,Alan,PIH,MON LADMC,1.5
6,Pope,Georgia,VPH,Mon LADMC,1.5
7,Kenady,Matthew,PIH,Thurs Gateways,1.0
8,de Los Arcos Diez,Pablo,PIH,Thurs Gateways,1.0
9,Yannuzzi,Amanda,Cedars,SAT GATEWAYS,0.5
